# Main explanation

#### Good lecture : https://www.cambridge.org/core/journals/journal-of-materials-research/article/double-perovskite-sr2bbo6-oxides-for-hightemperature-thermoelectric-power-generationa-review/87CFBEC724851ABAFEDB893AD7E747E8/core-reader

#### ISCD QUERY
One search on the ICSD was made for spacegroupe : Fm-3m with number of elements <=2 - 4776 queries
The second was made for spacegroupe : Fm-3m with number of elements >2 - 6123 queries
The third was made for spacegroupe : Fm-3m with number of elements >3 - 2037 queries

## Importation

In [30]:
import pandas as pd
import numpy as np
import sklearn as skl
import matplotlib as mp
import sys as os
import scipy as sp
import aiida

ModuleNotFoundError: No module named 'aiida'

In [28]:
df1 = pd.read_json("ISCD3-2/ISCD3_CollCode28065.cif")

ValueError: Expected object or value

## Openning from text file

In [27]:
df = pd.DataFrame(pd.read_table("ISCD3.txt")) #even thoug the data is already frammed
df1 = pd.read_json("ISCD3-2/ISCD3_CollCode28065.cif")
df.to_csv('ISCD333.csv')
df

,CollectionCode,HMS,StructuredFormula,StructureType,Title,CellParameter,ReducedCellParameter,StandardisedCellParameter,CellVolume,FormulaUnitsPerCell,...,MineralGroup,CalculatedDensity,MeasuredDensity,PearsonSymbol,WyckoffSequence,Journal,Volume,PublicationYear,Quality,Unnamed: 28
0,189,F m -3 m,Ba2 Mn W O6,Elpasolite#K2NaAlF6#Ba2CaWO6,Magnetic structure of Ba2 Mn W O6,8.193 8.193 8.193 90 90 90,5.7933 5.7933 5.7933 60.000 60.000 60.000,8.1930 8.1930 8.1930 90.000 90.000 90.000,549.96,4,...,NaN,7.36,NaN,cF40,e c b a,Journal of Solid State Chemistry,17,1976,-1,NaN
1,605,F m -3 m,(N H4)2 (Sn Cl6),K2PtCl6,Potassium hexachlorostannate(IV) and ammonium ...,10.038(2) 10.038(2) 10.038(2) 90 90 90,7.0979 7.0979 7.0979 60.000 60.000 60.000,10.0380 10.0380 10.0380 90.000 90.000 90.000,1011.44,4,...,NaN,2.41,NaN,cF68,e c a,"Acta Crystallographica, Section B: Structural ...",32,1976,10,NaN
2,1000,F m -3 m,Cs2 Li (Fe (C N)6),Cs2LiCo(CN)6,Crystal structure of Cs2 Li Fe (C N)6 by neutr...,10.571(1) 10.571(1) 10.571(1) 90 90 90,7.4748 7.4748 7.4748 60.000 60.000 60.000,10.5710 10.5710 10.5710 90.000 90.000 90.000,1181.27,4,...,NaN,2.73,NaN,cF64,e2 c b a,Inorganic Chemistry,16,1977,10,NaN
3,1001,F m -3 m,Cs2 Li (Fe (C N)6),Cs2LiCo(CN)6,Crystal structure of Cs2 Li Fe (C N)6 by neutr...,10.571(1) 10.571(1) 10.571(1) 90 90 90,7.4748 7.4748 7.4748 60.000 60.000 60.000,10.5710 10.5710 10.5710 90.000 90.000 90.000,1181.27,4,...,NaN,2.73,NaN,cF64,e2 c b a,Inorganic Chemistry,16,1977,10,NaN
4,1005,F m -3 m,Eu Ba2 Nb O5.5,Elpasolite(defect)#K2NaAlF6-x,Preparation and properties of new perovskite t...,8.49 8.49 8.49 90 90 90,6.0033 6.0033 6.0033 60.000 60.000 60.000,8.4900 8.4900 8.4900 90.000 90.000 90.000,611.96,4,...,NaN,6.59,6.25,cF38,e c b a,Inorganic Chemistry,16,1977,-1,NaN
5,1006,F m -3 m,Eu2 Ba Ta O5.5,Elpasolite(defect)#K2NaAlF6-x,Preparation and properties of new perovskite t...,8.394 8.394 8.394 90 90 90,5.9355 5.9355 5.9355 60.000 60.000 60.000,8.3940 8.3940 8.3940 90.000 90.000 90.000,591.43,4,...,NaN,7.98,7.98,cF38,e c b a,Inorganic Chemistry,16,1977,-1,NaN
6,1278,F m -3 m,Mn3 (Co (C N)6)2 (H2 O)13,Mn3(Co(CN)6)2(H2O)12,Crystal structure of Mn3 (Co (C N)6)2 (H2 O)1...,10.436(3) 10.436(3) 10.436(3) 90 90 90,7.3794 7.3794 7.3794 60.000 60.000 60.000,10.4360 10.4360 10.4360 90.000 90.000 90.000,1136.59,2,...,NaN,2.42,NaN,cF136,f e2 c b a,Inorganic Chemistry,16,1977,10,NaN
7,1279,F m -3 m,Mn3 (Co (C N)6)2 (H2 O)13,Mn3(Co(CN)6)2(H2O)12,Crystal structure of Mn3 (Co (C N)6)2 (H2 O)1...,10.436(3) 10.436(3) 10.436(3) 90 90 90,7.3794 7.3794 7.3794 60.000 60.000 60.000,10.4360 10.4360 10.4360 90.000 90.000 90.000,1136.59,2,...,NaN,2.42,NaN,cF136,f e2 c b a,Inorganic Chemistry,16,1977,-1,NaN
8,1280,F m -3 m,(Cd3 (Co (C N)6)2 (H2 O)13)1.333,Mn3(Co(CN)6)2(H2O)12,Crystal structure of Mn3 (Co (C N)6)2 (H2 O)1...,10.600(3) 10.600(3) 10.600(3) 90 90 90,7.4953 7.4953 7.4953 60.000 60.000 60.000,10.6000 10.6000 10.6000 90.000 90.000 90.000,1191.02,1,...,NaN,1.86,NaN,cF91,f e3 c b a,Inorganic Chemistry,16,1977,-1,NaN
9,1341,F m -3 m,(N H4)3 (Ti F6.98),(NH4)3TiF7,The disordered structure of ammonium pentafluo...,9.231(5) 9.231(5) 9.231(5) 90 90 90,6.5273 6.5273 6.5273 60.000 60.000 60.000,9.2310 9.2310 9.2310 90.000 90.000 90.000,786.59,4,...,NaN,1.98,NaN,cF92,j2 c b a,"Acta Chemica Scandinavica, Series A: Physical ...",31,1977,10,NaN


# Second method using shell

## Using ssh to connect to the server hosting the database 

In [2]:
!ssh -fN -L 3306:localhost:3306 -L 8010:localhost:80 aiida@theossrv2.epfl.ch > /dev/null 2>&1 #using username of someone from aiida team

## Loading the ICSD importer class and instantiate it

In [31]:
importer_class = DbImporterFactory('icsd')
importer_parameters = {'server': 'http://theossrv2.epfl.ch',
            'host': '127.0.0.1',
            'db': 'icsd',
            'passwd': 'sql'

importer = importer_class(**importer_parameters)
importer

SyntaxError: invalid syntax (<ipython-input-31-0e887010430e>, line 7)

In [32]:
importer_class = DbImporterFactory('icsd')

NameError: name 'DbImporterFactory' is not defined

## Query the ICSD for a given database entry 



In [ ]:
icsd_id = '617290' #graphite

query_results = importer.query(id=icsd_id)

query_results_perovskyte_type = importer.query(spacegroup='F m -3 m')
# ou Fm3m avec une barre sur le 3 (Fm-3m??)

# Hall notation : -F 4 2 3
# Hermang : F m -3 m 
# number : 225


print len(query_results)
print len(query_results_perovskyte_type)

## Get the cif file corresponding to the entry, and convert it to an AiiDA structure



In [ ]:
entre = query_results.at(0)
cif = entry.get_cif_node()
print cif.get_formulae()

structure = cif._get_aiida_structure(converter='pymatgen')
print structure.get_formula()
print structure.get_ase()

## Get the spacegroup of the structure with spglib



In [ ]:
print spglib.get_spacegroup(structure.get_ase())

## Store the structure and prints its pk



In [ ]:
structure.store()
structure.pk

### Now access the terminal and type the following to visualise the structure


#### verdi data structure show --format xcrysden 5084

